In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import PyCO2SYS as pyco2
import xarray as xr
from datetime import datetime
from scipy.optimize import minimize

In [73]:
# import data from csv file
data = pd.read_csv('carbon_species_alk_dic_table.csv', usecols=[0, 1])

# save first column as alkalinity
alkalinity = data.iloc[:, 0].values
# save second column as DIC
dic = data.iloc[:, 1].values


In [74]:
print("Alkalinity:", alkalinity)
print("DIC:", dic)

# If you have constant temperature and salinity, set them here:
temperature = np.full_like(dic, 19.0)  # degrees Celsius
salinity = np.full_like(dic, 35.0)     # PSU

Alkalinity: [2321.34753  2321.34753  2321.34753  2321.34753  2321.34753  2321.34753
 2321.34753  2321.34753  2321.34753  2321.34753  2321.34753  2321.34753
 2321.34753  2321.34753  2321.34753  2321.34753  2321.34753  2321.34753
 2321.34753  2321.34753  2321.34753  2321.34753  2417.687397 2417.687397
 2417.687397 2417.687397 2417.687397 2417.687397 2417.687397 2417.687397
 2417.687397 2417.687397 2417.687397 2417.687397 2417.687397 2417.687397
 2417.687397 2417.687397 2417.687397 2417.687397 2417.687397 2417.687397
 2417.687397 2417.687397]
DIC: [2019.05574   2029.8126195 2040.569499  2051.3263785 2062.083258
 2072.8401375 2083.597017  2094.3538965 2105.110776  2115.8676555
 2126.624535  2137.3814145 2148.138294  2158.8951735 2169.652053
 2180.4089325 2191.165812  2201.9226915 2212.679571  2223.4364505
 2234.19333   2244.9502095 2019.05574   2029.8126195 2040.569499
 2051.3263785 2062.083258  2072.8401375 2083.597017  2094.3538965
 2105.110776  2115.8676555 2126.624535  2137.3814145 214

In [75]:
# Call PyCO2SYS
results = pyco2.sys(
    par1=alkalinity,
    par2=dic,
    par1_type=1,   # 1 = total alkalinity
    par2_type=2,   # 2 = DIC
    salinity=salinity,
    temperature=temperature,
    opt_k_carbonic=11,  # Mojica Prieto, F. J., and Millero, F. J. (2002). The values of pK1 + pK2 for the dissociation of carbonic acid in seawater. Geochimica et Cosmochimica Acta 66, 2529–2540. doi:10.1016/S0016-7037(02)00855-4.
)

In [76]:
# Extract desired outputs
carbonate = results['CO3']           # µmol/kg
bicarbonate = results['HCO3']        # µmol/kg
pco2 = results['pCO2']               # µatm
ph = results['pH']                   # pH on total scale

# Optionally, add results to your DataFrame
data['Temperature'] = temperature
data['Salinity'] = salinity
data['Carbonate'] = carbonate
data['Bicarbonate'] = bicarbonate
data['pCO2'] = pco2
data['pH'] = ph


# Save to CSV
data.to_csv('carbon_species_results_prieto.csv', index=False)